In [90]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [92]:
!kaggle datasets download -d thedevastator/higher-education-predictors-of-student-retention
!unzip higher-education-predictors-of-student-retention.zip



Dataset URL: https://www.kaggle.com/datasets/thedevastator/higher-education-predictors-of-student-retention
License(s): CC0-1.0
higher-education-predictors-of-student-retention.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  higher-education-predictors-of-student-retention.zip
replace dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset.csv             


In [93]:
import pandas as pd
import numpy as np

df = pd.read_csv("dataset.csv")  # upload your dataset to Colab
print(df.shape)
df.head()


(4424, 35)


,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,8,5,2,1,1,1,13,10,6,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,6,1,11,1,1,1,1,3,4,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,5,1,1,1,22,27,10,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,8,2,15,1,1,1,23,27,6,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,12,1,3,0,1,1,22,28,10,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [94]:
df["Target_binary"] = df["Target"].replace({
    "Dropout": 0,
    "Enrolled": 1,
    "Graduate": 1
})

df["Target_binary"].value_counts()


/tmp/ipython-input-2370890742.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Target_binary"] = df["Target"].replace({


,count
Target_binary,
1,3003
0,1421


In [95]:
categorical_features = [
    'Gender',
    'Marital status',
    'Course',
    'Scholarship holder',
    'Tuition fees up to date',
    'Debtor',
    "Mother's qualification",
    "Father's qualification"
]


In [96]:
numerical_features = [
    'Age at enrollment',
    'Curricular units 1st sem (enrolled)',
    'Curricular units 1st sem (evaluations)',
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (credited)',
    'Application order'
]


In [97]:
features = categorical_features + numerical_features
X = df[features]
y = df["Target_binary"]

print(X.shape, y.shape)


(4424, 14) (4424,)


In [98]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", StandardScaler(), numerical_features)
    ]
)

X_processed = preprocessor.fit_transform(X)
print("Processed shape:", X_processed.shape)


Processed shape: (4424, 100)


In [99]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [100]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weights = dict(enumerate(class_weights))
print(class_weights)


{0: np.float64(1.5562884784520667), 1: np.float64(0.7366777685262281)}


In [101]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation="relu"),
    BatchNormalization(),
    Dropout(0.2),

    Dense(1, activation="sigmoid")  # binary output
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_23 (Dense)                │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,017 (86.00 KB)

 Trainable params: 21,633 (84.50 KB)

 Non-trainable params: 384 (1.50 KB)

In [102]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    class_weight=class_weights
)


Epoch 1/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.6588 - loss: 0.6712 - val_accuracy: 0.8554 - val_loss: 0.4847
Epoch 2/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8056 - loss: 0.4469 - val_accuracy: 0.8497 - val_loss: 0.4041
Epoch 3/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8242 - loss: 0.4095 - val_accuracy: 0.8520 - val_loss: 0.3682
Epoch 4/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8186 - loss: 0.4248 - val_accuracy: 0.8486 - val_loss: 0.3614
Epoch 5/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8307 - loss: 0.4068 - val_accuracy: 0.8486 - val_loss: 0.3568
Epoch 6/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8297 - loss: 0.4016 - val_accuracy: 0.8542 - val_loss: 0.3512
Epoch 7/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8317 - loss: 0.4015 - val_accuracy: 0.8463 - val_loss: 0.3554
Epoch 8/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8402 - loss: 0.3878 - val_accu

In [103]:
from sklearn.metrics import classification_report, confusion_matrix

loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["Dropout", "Success"]))


28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8642 - loss: 0.3507
Test Accuracy: 0.8542372584342957
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
[[211  73]
 [ 56 545]]
              precision    recall  f1-score   support

     Dropout       0.79      0.74      0.77       284
     Success       0.88      0.91      0.89       601

    accuracy                           0.85       885
   macro avg       0.84      0.82      0.83       885
weighted avg       0.85      0.85      0.85       885



In [107]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC-AUC:", roc_auc)


ROC-AUC: 0.9060310280987087


In [104]:
import joblib

model.save("student_success_binary_model.h5")
joblib.dump(preprocessor, "preprocessor.pkl")

!ls


dataset.csv
drive
higher-education-predictors-of-student-retention.zip
label_encoder.pkl
preprocessor.pkl
sample_data
student_success_binary_model.h5
student_success_model.h5


In [108]:
feature_names = preprocessor.get_feature_names_out()
joblib.dump(feature_names, "feature_names.pkl")


['feature_names.pkl']

In [109]:
from google.colab import files

files.download("student_success_binary_model.h5")
files.download("preprocessor.pkl")
files.download("feature_names.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>